In [1]:
import pandas as pd
import torch
import torch.nn as nn
import transformers
from transformers import BertModel
from transformers import AutoTokenizer
from torch.utils.data import Dataset
# from transformers import AutoModelForSequenceClassification
from torch.utils.data import DataLoader, TensorDataset
print(transformers.__version__)
# !pip install evaluate
import evaluate
import os
import natsort
from PIL import Image
import numpy as np

4.15.0


2023-07-15 17:28:01.345396: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


## Data Preparation

In [2]:
file_path="/home/nitesh/Documents/MY_THESIS/MAMI"

train_filename="training1.csv"
train_data_file=os.path.join(file_path, train_filename)
train2 = pd.read_csv(train_data_file, delimiter='\t')

IDs = [] 
images = []
directory = os.path.join(file_path, 'TRAINING')   # directory where we have images 
filenames = natsort.natsorted(os.listdir(directory))  

# get the ids from the images, where images are having three channels; omit images if channels != 3
for i, filename in enumerate(filenames):
#     print(i, filename)
    if filename.endswith(".jpg"):
#         ID = int(filename[:-4])
        ID = filename
        pathname = os.path.join(directory, filename)
        im = Image.open(pathname)
        im = im.resize((224, 224))  # Resize the image to (224, 224)
        imnp = np.array(im)
        if len(imnp.shape) != 3:
#             print("This is 1 channel, so we omit it", imnp.shape, filename)
            continue
        IDs.append(ID)
        images.append(imnp)

def get_common_strings(list1, list2):
    return list(set(list1) & set(list2))

# Example usage
list1 = IDs
list2 = list(train2.file_name)  #from the text file where we have text description 
common_strings = get_common_strings(list1, list2)
print('len of common strings', len(common_strings))

sorted_ids = natsort.natsorted(common_strings)

# print(sorted_ids)

# Sort the dataframe with natural ordering of the IDs
train2['prefix_file_name'] = train2['file_name'].str.extract('(\d+)').astype(int)
# Assuming 'df' is your DataFrame
sorted_train_df = train2.sort_values(by='prefix_file_name', ascending=True)
sorted_train_df

# Assuming 'df' is your DataFrame and 'common_strings' is the list of strings
# Get the common string values in the column 
filtered_df = sorted_train_df[sorted_train_df['file_name'].isin(sorted_ids)].reset_index(drop=True)

# Print the filtered DataFrame
print('filtered_df shape', filtered_df.shape)

train3 = filtered_df.copy()
# print(train3.shape)

trainx = train3.rename(columns={'Text Transcription': 'text'})
# display(trainx)


### LOAD TEST DATA

test_filename="Test.csv"
test_data_file=os.path.join(file_path, test_filename)
test1 = pd.read_csv(test_data_file, delimiter='\t')


test_labels_filename = 'test_labels.txt'
test_lbls_file = os.path.join(file_path, test_labels_filename)
test_labels = pd.read_csv(test_lbls_file, 
                          delimiter='\t',
                          header=None)

test_labels.columns = ['file_name', 
                      "misogynous",
                       "shaming",
                       "stereotype",
                       "objectification",
                       "violence"]

merged_test = pd.merge(test1, test_labels, on='file_name', how='inner')

# Sort the dataframe with natural ordering of the IDs
merged_test['prefix_file_name'] = merged_test['file_name'].str.extract('(\d+)').astype(int)
# Assuming 'df' is your DataFrame
merged_test1 = merged_test.sort_values(by='prefix_file_name', ascending=True)
merged_test1


# # train = train.rename(columns={'Text Transcription': 'text'})
test2 = merged_test1.rename(columns={'Text Transcription': 'text'})

train = trainx[['file_name', 'text', 'misogynous']]
train = train.rename(columns = {'misogynous':'label'})

test = test2[['file_name', 'text', 'misogynous']]
test = test.rename(columns = {'misogynous':'label'})


import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

labelencoder = LabelEncoder()
labelencoder.fit(train['label'])
train['encoded_label'] = labelencoder.fit_transform(train['label'])
test["encoded_label"] = labelencoder.transform(test['label'])
display(train)
display(test)

len of common strings 9986
filtered_df shape (9986, 8)


,file_name,text,label,encoded_label
0,1.jpg,Milk Milk.zip,0,0
1,2.jpg,-What are you doing? -you told me to satanize ...,0,0
2,3.jpg,imgflip.com ME 1254 NEW BUGS AFTER CHANGES BUG...,0,0
3,4.jpg,Bedroom Kitchen Bathroom Bron memes storage,0,0
4,5.jpg,WAKEUP EARLY FREELANCERS,0,0
...,...,...,...,...
9981,15002.jpg,WAITING FOR THE END OF THE COVID imgflip.com,0,0
9982,15003.jpg,SMART WOMEN ARE AROUND imgflip.com,0,0
9983,15004.jpg,GOOD GIRLS ARE BEHIND THE CORNER imgflip.com,0,0
9984,15005.jpg,COOKING FOR MY WIFE imgflip.com,0,0


,file_name,text,label,encoded_label
436,15001.jpg,G HIS. UNDYING FIDELITY Steve is hot and perfe...,0,0
854,15002.jpg,How limagined myself as a Teacher...... How I ...,0,0
743,15004.jpg,WHERE WILL YOU BE WHEN DIARRHEA STRIKE memecen...,0,0
44,15005.jpg,A MAN WITH DREAMS... NEEDS A WOMAN WITH VISION,0,0
873,15006.jpg,THIS IS HOW YOUR GIRLFRIEND SEES YOUR FEMALE F...,0,0
...,...,...,...,...
871,17078.jpg,There are multiple reasons to lower your car T...,0,0
683,17079.jpg,MICHELLE OBAMA IS A MAN IGUARANTEE IT makeamem...,1,1
406,17080.jpg,Looks like the airbags deployed 1234498,0,0
733,17081.jpg,Half woman half horse,0,0


## Model Training 

In [3]:
import random
import tensorflow as tf
PERFORMANCE_RESULTS = pd.DataFrame(columns=['Model', 'EvalSet', 'Accuracy', 
                       'M-Precision', 'M-Recall', 
                       'M-F1-Score', 'W-Precision',
                       'W-Recall', 'W-F1-Score'])

for KK in list([7,10,18,22,55,77]): 

    random.seed(KK)
    np.random.seed(KK)
    torch.manual_seed(KK)
#     tf.random.set_seed(KK)

# KK=7
    train_labels = torch.tensor(train["encoded_label"].tolist())
    test_labels = torch.tensor(test["encoded_label"].tolist())

    # Convert your train text data to a list
    train_texts = train["text"].tolist()
    test_texts = test["text"].tolist()

    # Initialize the tokenizer
    tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")



    train_encodings = tokenizer(
        train_texts,
        padding=True,           # pad all inputs to max length
        max_length=12,         # Bert max is 512, we choose 24 for computational efficiency
        return_tensors="pt",    # Return format pytorch tensor
        truncation=True
    )

    print('train encodings shape ', train_encodings['input_ids'].shape)


    test_encodings = tokenizer(
        test_texts,
        padding=True,           # pad all inputs to max length
        max_length=12,         # Bert max is 512, we choose 24 for computational efficiency
        return_tensors="pt",    # Return format pytorch tensor
        truncation=True
    )

    print('test encodings shape ', test_encodings['input_ids'].shape)


    # Convert the tokenized inputs into a PyTorch dataset
    train_ds = TensorDataset(
        train_encodings["input_ids"],
        train_encodings["attention_mask"],
        train_labels  # Assuming you have the corresponding train labels
    )


    # Convert the tokenized inputs into a PyTorch dataset
    test_ds = TensorDataset(
        test_encodings["input_ids"],
        test_encodings["attention_mask"],
        test_labels  # Assuming you have the corresponding train labels
    )


    # Define batch size for the train_loader and test_loader
    batch_size = 16

    # Create the train_loader
    train_loader = DataLoader(
        train_ds,
        batch_size=batch_size,
        shuffle=True
    )

    # Create the test_loader
    # for i in test_loader:
    #     print(i)

    test_loader = DataLoader(
        test_ds,
        batch_size=batch_size,
        shuffle=False
    )


    from transformers import AutoModelForSequenceClassification

    base_model1 = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

    device = 'cpu'

    # # Set the device for training
#     device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model1 = base_model1.to(device)

    from torch.optim import AdamW
    from tqdm.auto import tqdm
    # Define your loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = AdamW(model1.parameters(), lr=5e-5)

    # Set the number of epochs and calculate the total training steps
    num_epochs = 8
    num_training_steps = num_epochs * len(train_loader)

    # Training loop
    model1.train()
    progress_bar = tqdm(total=num_training_steps)


    for epoch in range(num_epochs):

        epoch_loss = 0.0 

        for batch in train_loader:
            input_ids, attention_mask, labels = batch
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            labels = labels.to(device)

            # Forward pass
            outputs = model1(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss

            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()  # Accumulate the loss
    #         print('Loss item', loss.item())

            progress_bar.update(1)
            progress_bar.set_description(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.4f}")

        avg_loss = epoch_loss / len(train_loader)  # Calculate average loss for the epoch
        print(f"Epoch {epoch+1}/{num_epochs}, Average Loss: {avg_loss:.4f}")

    progress_bar.close()


    from sklearn.metrics import classification_report

    # Evaluation loop
    model1.eval()

    predictions = []
    true_labels = []

    # Disable gradient calculation
    with torch.no_grad():
        for batch in test_loader:
            input_ids, attention_mask, labels = batch
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            labels = labels.to(device)

            # Forward pass
            outputs = model1(input_ids, attention_mask=attention_mask)
            logits = outputs.logits

            # Get predicted labels
    #         _, predicted = torch.max(logits, dim=1)
            predicted = torch.argmax(logits, dim=-1)

            predictions.extend(predicted.cpu().tolist())
            true_labels.extend(labels.cpu().tolist())
            


    ## Generate Classification Report
    def export_classification_report(report, mod, eval_set=None):
        lines = report.split('\n')
        data = lines[2:4] + lines[5:9]
        data = [line.split() for line in data]

        acc = float(data[2][1])
        m_prec = float(data[3][2])
        m_recall = float(data[3][3])
        m_f1 = float(data[3][4])
        w_prec = float(data[4][2])
        w_recall = float(data[4][3])
        w_f1 = float(data[4][4])

        df = pd.DataFrame({
            'Model': [mod],
            'EvalSet': [eval_set],
            'Accuracy': [acc],
            'M-Precision': [m_prec],
            'M-Recall': [m_recall],
            'M-F1-Score': [m_f1],
            'W-Precision': [w_prec],
            'W-Recall': [w_recall],
            'W-F1-Score': [w_f1],
        }) 

        return df

    # Calculate classification report
    classification_rep = classification_report(true_labels, predictions, digits=3)

    # Print classification report
    print(classification_rep)

    modname  = 'FineTuned_BERT'
    results_test= export_classification_report(classification_rep, modname, eval_set='test')

    PERFORMANCE_RESULTS = pd.concat([PERFORMANCE_RESULTS, results_test])

    #     display(PERFORMANCE_RESULTS)


display(PERFORMANCE_RESULTS)


train encodings shape  torch.Size([9986, 12])
test encodings shape  torch.Size([1000, 12])


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

  0%|          | 0/5000 [00:00<?, ?it/s]

Epoch 1/8, Average Loss: 0.5628
Epoch 2/8, Average Loss: 0.4142
Epoch 3/8, Average Loss: 0.2342
Epoch 4/8, Average Loss: 0.1474
Epoch 5/8, Average Loss: 0.0883
Epoch 6/8, Average Loss: 0.0741
Epoch 7/8, Average Loss: 0.0607
Epoch 8/8, Average Loss: 0.0586
              precision    recall  f1-score   support

           0      0.691     0.518     0.592       500
           1      0.614     0.768     0.683       500

    accuracy                          0.643      1000
   macro avg      0.653     0.643     0.637      1000
weighted avg      0.653     0.643     0.637      1000

train encodings shape  torch.Size([9986, 12])
test encodings shape  torch.Size([1000, 12])


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

  0%|          | 0/5000 [00:00<?, ?it/s]

Epoch 1/8, Average Loss: 0.5601
Epoch 2/8, Average Loss: 0.4094
Epoch 3/8, Average Loss: 0.2428
Epoch 4/8, Average Loss: 0.1396
Epoch 5/8, Average Loss: 0.0930
Epoch 6/8, Average Loss: 0.0776
Epoch 7/8, Average Loss: 0.0598
Epoch 8/8, Average Loss: 0.0532
              precision    recall  f1-score   support

           0      0.712     0.480     0.573       500
           1      0.608     0.806     0.693       500

    accuracy                          0.643      1000
   macro avg      0.660     0.643     0.633      1000
weighted avg      0.660     0.643     0.633      1000



ConnectionError: HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /api/models/bert-base-uncased (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f32d2c693a0>: Failed to establish a new connection: [Errno -3] Temporary failure in name resolution'))

In [ ]:
PERFORMANCE_RESULTS.to_excel('../MAMI/9_classification_report_Finetuning_BERT.xlsx', 
                      index=False)

https://huggingface.co/docs/transformers/training
https://mccormickml.com/2019/07/22/BERT-fine-tuning/

https://huggingface.co/docs/transformers/main_classes/trainer#transformers.TrainingArguments
https://pytorch.org/docs/stable/generated/torch.nn.Linear.html

https://huggingface.co/blog/fine-tune-vit